In [2]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import *

print('SAGA code version', SAGA.__version__)

SAGA code version 0.6.1


In [3]:
# import other modules

from collections import defaultdict
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from easyquery import Query

In [4]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)

saga_database = SAGA.Database('/home/yymao/Dropbox/Academia/Collaborations/SAGA', '/home/yymao/Documents/Research/SAGA')
saga_host_catalog = SAGA.HostCatalog(saga_database)
saga_object_catalog = SAGA.ObjectCatalog(saga_database)

In [7]:
is_sat = (C.is_sat & C.is_clean)
is_lowz = (C.is_low_z & C.is_clean & C.has_spec)

def get_counts(base_list):
    data = defaultdict(list)

    for t in base_list:
        host_id = int(t['HOST_NSAID'][0])

        data['SAGA name'].append(saga_host_catalog.id_to_name(host_id))
        data['NSAID'].append(host_id)
        
        data['sat'].append(is_sat.count(t))
        data['low_z'].append(is_lowz.count(t))
        data['total'].append(len(t))

        for i in (-1, 0, 1, 2, 3, 4):
            data['zq_{}'.format(i)].append(Query('ZQUALITY=={}'.format(i)).count(t))

        for i in (-1, 0, 1, 2, 3, 4, 5):
            data['rm_{}'.format(i)].append(Query('REMOVE=={}'.format(i)).count(t))

    return Table(data)

## load base catalogs

In [6]:
# specify columns to load
columns = 'OBJID HOST_NSAID RA DEC SPEC_Z REMOVE SATS ZQUALITY TELNAME SPEC_REPEAT r_mag OBJ_NSAID coord'.split()
cuts = C.is_galaxy & C.fibermag_r_cut

# load base catalogs (will take a while)
catalogs_orig = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list', version='paper1')
catalogs_dr14 = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list', version=1)

## tables of counts

In [8]:
# Current base catalogs
get_counts(catalogs_orig)

SAGA name,NSAID,sat,low_z,total,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Bandamanna,126115,1,4,26788,26140,0,12,0,24,612,23871,0,16,26,1890,24,961
HarryPotter,129237,4,6,27415,26723,0,52,16,41,583,24554,0,10,59,1708,23,1061
Narnia,132339,2,5,39272,36935,0,146,20,212,1959,35998,0,6,203,2150,104,811
StarTrek,33446,2,5,35451,33578,0,102,8,124,1639,32015,0,9,55,2478,102,792
MobyDick,85746,1,6,40506,38750,0,67,7,80,1602,37574,0,24,70,1606,72,1160
Oz,137625,5,9,33426,32607,0,21,8,25,765,31018,0,9,78,1358,40,923
Alice,140594,2,4,39222,36470,0,71,11,151,2519,36332,0,9,142,1542,111,1086
Sopranos,13927,0,2,36092,34422,0,34,1,41,1594,32738,0,6,291,1816,123,1118
ScoobyDoo,161174,4,8,35655,34573,0,37,2,43,1000,33441,0,2,71,1316,63,762


In [9]:
# new base catalogs with DR14
get_counts(catalogs_dr14)

SAGA name,NSAID,sat,low_z,total,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Bandamanna,126115,1,4,26769,26017,0,61,0,24,667,24617,0,16,29,1295,812,0
HarryPotter,129237,4,7,27350,26603,0,62,16,41,628,25293,0,20,78,1351,608,0
Narnia,132339,2,5,39222,36709,0,176,22,213,2102,36573,0,11,216,1639,783,0
StarTrek,33446,2,5,35431,33559,0,105,8,123,1636,32617,0,14,59,2128,613,0
MobyDick,85746,1,7,40438,38682,0,66,7,81,1602,38456,0,37,77,1307,561,0
Oz,137625,5,9,33540,32720,0,21,8,25,766,31862,0,10,81,1126,461,0
Alice,140594,2,5,39137,36150,0,69,10,157,2609,37138,0,15,146,1155,683,0
Sopranos,13927,0,2,36061,34335,0,31,1,47,1493,33547,0,19,322,1305,868,0
ScoobyDoo,161174,4,8,35884,34806,0,33,2,43,1000,34276,0,7,75,1058,468,0


## check satellites

In [10]:
vstack([is_sat.filter(t)[t.colnames[:-1]] for t in catalogs_dr14]).show_in_notebook()

idx,OBJID,HOST_NSAID,RA,DEC,SPEC_Z,REMOVE,SATS,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,OBJ_NSAID
0,1237680247351738669,126115,1.129657963531551,20.71801839607005,0.0072550303,-1,1,4,NSA,NSA+SDSS,15.868438,126134
1,1237678602387456305,129237,20.328466622978624,17.7539390552514,0.008245,-1,1,4,MMT,MMT,19.62673,-1
2,1237678602387521791,129237,20.536342698008013,17.52814287890618,0.008651,-1,1,4,MMT,MMT,18.976677,-1
3,1237678602387456130,129237,20.28526901056989,17.602235752487275,0.00822,-1,1,4,MMT,MMT,17.315659,-1
4,1237679169841725673,129237,20.77689036929064,17.891450173095595,0.008477,-1,1,4,MMT,MMT,17.114355,-1
5,1237678881574814166,132339,39.53616133150837,-1.3966955912552497,0.00845,-1,1,3,AAT,AAT,20.117296,-1
6,1237678881574944900,132339,39.92546425206123,-1.4187419100030116,0.008789,-1,1,4,MMT,MMT,16.977049,-1
7,1237657607497515484,33446,123.64989755576431,36.43435539786629,0.008211395,-1,1,4,NSA,NSA+SDSS,16.393507,33448
8,1237657607497318756,33446,123.24317320594254,36.19835996595335,0.008303763,-1,1,4,NSA,NSA+SDSS,15.436748,33445
9,1237664338780029261,85746,149.68716885916922,32.7202654616559,0.004987,-1,1,3,MMT,MMT,20.664415,-1


## difference in low-z galaxy count is due to whether or not host is counted

In [11]:
for to, tn in zip(catalogs_orig, catalogs_dr14):
    # make sure we are comparing the same host
    assert to['HOST_NSAID'][0] == tn['HOST_NSAID'][0]
    
    # check if satellite number is the same
    if is_sat.count(to) != is_sat.count(tn):
        print('different satellite count for {}'.format(tn['HOST_NSAID'][0]))
    
    if is_lowz.count(to) != is_lowz.count(tn):
        print('different low-z galaxy count for {}: {} (orig) vs. {} (new)'.format(\
                    tn['HOST_NSAID'][0], is_lowz.count(to), is_lowz.count(tn)))
        tn_lowz = is_lowz.filter(tn)
        to_lowz = is_lowz.filter(to)
        for gal in tn_lowz:
            if len(find_near_coord(to_lowz, gal['coord'], 1.0)) == 0:
                print(' missing obj NSAID', gal['OBJ_NSAID'])

different low-z galaxy count for 129237: 6 (orig) vs. 7 (new)
 missing obj NSAID 129127
different low-z galaxy count for 85746: 6 (orig) vs. 7 (new)
 missing obj NSAID -1
different low-z galaxy count for 140594: 4 (orig) vs. 5 (new)
 missing obj NSAID -1
different low-z galaxy count for 165536: 8 (orig) vs. 9 (new)
 missing obj NSAID 145456
different low-z galaxy count for 145729: 4 (orig) vs. 5 (new)
 missing obj NSAID 145729
different low-z galaxy count for 166313: 9 (orig) vs. 10 (new)
 missing obj NSAID -1
different low-z galaxy count for 147100: 14 (orig) vs. 15 (new)
 missing obj NSAID 147100
different low-z galaxy count for 149781: 4 (orig) vs. 5 (new)
 missing obj NSAID 149781
different low-z galaxy count for 150887: 11 (orig) vs. 12 (new)
 missing obj NSAID 150793
different low-z galaxy count for 61945: 7 (orig) vs. 8 (new)
 missing obj NSAID 151978


## checking some remove-4 flag objects

In [12]:
host_nsa = 166313
columns = 'OBJID HOST_NSAID RA DEC SPEC_Z REMOVE SATS ZQUALITY TELNAME SPEC_REPEAT PETRORAD_R PETRORADERR_R r_mag OBJ_NSAID coord'.split()
bo = saga_object_catalog.load(host_nsa, columns=columns, version='paper1').pop()
bn = saga_object_catalog.load(host_nsa, columns=columns, version=1).pop()

In [13]:
bn_objid_rm4 = Query('REMOVE==4').filter(bn)['OBJID']
bo_objid_rm4 = Query('REMOVE==4').filter(bo)['OBJID']

kn = bn_objid_rm4[np.in1d(bn_objid_rm4, bo_objid_rm4, invert=True)]
ko = bo_objid_rm4[np.in1d(bo_objid_rm4, bn_objid_rm4, invert=True)]

print(len(bn_objid_rm4), len(bo_objid_rm4))
print(len(kn), len(ko))

6820 275
6569 24


In [14]:
i = kn[1]
display(find_near_objid(bo, i, 15))
display(find_near_objid(bn, i, 15))

OBJID,HOST_NSAID,RA,DEC,SPEC_Z,REMOVE,SATS,ZQUALITY,TELNAME,SPEC_REPEAT,PETRORAD_R,PETRORADERR_R,r_mag,OBJ_NSAID,coord
,,,,,,,,,,,,,,"deg,deg"
int64,int64,float64,float64,float32,int64,int64,int64,str6,str48,float32,float32,float32,int64,object
1237667783928972103,166313,234.35127012297647,16.052379222497414,-1.0,3,-1,-1,,,2.9699788,-1000.0,25.840467,-1,"234.35127012297647,16.052379222497414"
1237667783928972104,166313,234.3517210725633,16.052247658020054,-1.0,3,-1,-1,,,2.9699788,-1000.0,26.05402,-1,"234.3517210725633,16.052247658020054"
1237667783928972095,166313,234.34853313842473,16.053177738827582,-1.0,5,-1,-1,,,2.9699788,-1000.0,27.31996,-1,"234.34853313842473,16.053177738827582"
1237667783928972107,166313,234.35391231754926,16.055008305275233,-1.0,-1,-1,-1,,,1.4076986,0.37581685,21.322119,-1,"234.35391231754926,16.055008305275233"


OBJID,HOST_NSAID,RA,DEC,SPEC_Z,REMOVE,SATS,ZQUALITY,TELNAME,SPEC_REPEAT,PETRORAD_R,PETRORADERR_R,r_mag,OBJ_NSAID,coord
,,,,,,,,,,,,,,"deg,deg"
int64,int32,float64,float64,float32,int16,int16,int16,str6,str48,float32,float32,float32,int32,object
1237667783928972103,166313,234.35127012297647,16.052379222497414,-1.0,4,-1,-1,,,2.9699788,-1000.0,25.85856,-1,"234.35127012297647,16.052379222497414"
1237667783928972104,166313,234.3517210725633,16.052247658020054,-1.0,4,-1,-1,,,2.9699788,-1000.0,26.07203,-1,"234.3517210725633,16.052247658020054"
1237667783928972095,166313,234.34853313842473,16.053177738827582,-1.0,3,-1,-1,,,2.9699788,-1000.0,27.337814,-1,"234.34853313842473,16.053177738827582"
1237667783928972107,166313,234.35391231684002,16.055008306546732,-1.0,-1,-1,-1,,,1.4076986,0.37581685,21.343405,-1,"234.35391231684002,16.055008306546732"
